In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw

In [2]:
geo = SplineGeometry()
m = 0.6
geo.AddCircle(c=(0, 0), r=1, bc="cyl", maxh=m) # 0.04
mesh = Mesh(geo.GenerateMesh(maxh=m))
mesh2 = Mesh(geo.GenerateMesh(maxh=m))
mesh.Curve(3);
mesh2.Curve(3)
scene1 = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [3]:
# vector space for shape gradient
VEC = H1(mesh, order=1, dim=2)
# Test and trial functions
PHI, X = VEC.TnT()
gfX = GridFunction(VEC)

In [4]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

dJOmega  = LinearForm(VEC)
vol = Parameter(1)
vol.Set(Integrate(surf_t,mesh))
alpha0 = 1
alpha = Parameter(alpha0)
#dJOmega += ((alpha*((vol-surf_0)**2)*dx)).DiffShape(X) # (definedon="mesh_1")
dJOmega += ((2*alpha*((vol-surf_0))*vol)*dx).DiffShape(X)
# vol-surf_0 = 1*dx() - zahl
# dJOmega += (surf_t*dx(definedon="mesh")-surf_t*dx(definedon="mesh2"))*((div(X))*dx)
# dJOmega += (surf_t*dx(definedon="mesh")-Parameter(surf_0))*((div(X))*dx)
#dJOmega += CoefficientFunction((x*y)).DiffShape(PSI)*dx

In [5]:
dJOmega.Assemble()
print(surf_0)
print(dJOmega.vec[0:5])

3.1417066211235207
       0
      0

       0
      0

       0
      0

       0
      0

       0
      0




In [6]:
mesh.UnsetDeformation()
test = CoefficientFunction((2*x,0.5*x))
gfX.Set(test)
mesh.SetDeformation(gfX)
scene = Draw(mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': None…

In [7]:
vol.Set(Integrate(surf_t,mesh))
dJOmega.Assemble()
print(vol.Get())
print(abs(surf_0-vol.Get()))
print(dJOmega.vec)

9.452133423394868
6.310426802271347
  30.299
 -180.824

  59.6283
 -8.6476e-15

  -30.299
 180.824

  -59.6283
 -3.65391e-14

  57.4223
 -160.421

  68.9504
 -92.2063

  38.2028
 92.2063

  3.99998
 160.421

  -57.4223
 160.421

  -68.9504
 92.2063

  -38.2028
 -92.2063

  -3.99998
 -160.421

  -1.06581e-14
 -2.84217e-14

  -1.42109e-14
      0

  -7.10543e-15
 -6.66134e-16

  1.02141e-14
 1.42109e-14

       0
 -1.35447e-14

  3.77476e-15
 -1.42109e-14

  7.10543e-15
 -2.84217e-14




## try to do this as shape optimization

In [8]:
gfset = GridFunction(VEC)

b = BilinearForm(VEC)
b += (InnerProduct(grad(X),grad(PHI))).Compile() *dx + (InnerProduct(X,PHI)).Compile()*dx

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [9]:
# volume before
Integrate(surf_t,mesh)

9.452133423394868

In [10]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
iter_max = 1

for i in range(0,iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    
    b.Assemble()
    vol.Set(Integrate(surf_t,mesh)) # needed for multiple iterations (already done once before)
    dJOmega.Assemble()
    SolveDeformationEquation()
    
    mesh.UnsetDeformation()
    
    gfxnorm = Norm(gfX.vec)
    scale = 0.01 / gfxnorm
    gfset.vec.data -= scale * gfX.vec
    Redraw(blocking=True)

In [11]:
# volume after
Integrate(surf_t,mesh)

3.1417066211235207